5.1 层和块

In [60]:
import torch 
from torch import nn 
from torch.nn import functional as F 


In [61]:
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.1837, -0.1259, -0.0433, -0.1111,  0.2664,  0.1865,  0.0885, -0.1800,
          0.0305, -0.0612],
        [-0.3189, -0.0474, -0.0756, -0.0656,  0.1825,  0.0503,  0.1392, -0.1241,
          0.1651, -0.0366]], grad_fn=<AddmmBackward0>)

##### 5.1 自定义块

In [62]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [71]:
class MySequential(nn.Module):
    def __init__(self,*args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module 
    
    def forward(self,X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X

In [72]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[-0.4346, -0.0175,  0.0320, -0.0988,  0.0980,  0.2625,  0.1872,  0.1381,
         -0.0808,  0.2150],
        [-0.2958,  0.0287, -0.0820, -0.0989,  0.0969,  0.3282,  0.0469,  0.1339,
          0.0310,  0.1157]], grad_fn=<AddmmBackward0>)

In [12]:
class FixHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=False)
        self.linear = nn.Linear(20,20)
    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [37]:
net = FixHiddenMLP()
net(X)

tensor(0.1541, grad_fn=<SumBackward0>)

In [38]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixHiddenMLP())
chimera(X)

tensor(0.0658, grad_fn=<SumBackward0>)

### 参数管理

In [39]:
import torch 
from torch import nn 

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand((2,4))
net(X)

tensor([[-0.2634],
        [-0.2317]], grad_fn=<AddmmBackward0>)

In [43]:
print(net[0].state_dict())

OrderedDict([('weight', tensor([[ 0.2562, -0.1817,  0.3075,  0.4317],
        [-0.2782, -0.3779,  0.1939, -0.0447],
        [-0.4222, -0.3913,  0.4981,  0.0049],
        [-0.2761,  0.1297,  0.3203,  0.4884],
        [ 0.0531, -0.0603, -0.0050, -0.4934],
        [ 0.1958, -0.4537,  0.0222,  0.0187],
        [-0.1292,  0.4046, -0.1512,  0.2520],
        [-0.2956, -0.0623,  0.1045,  0.4668]])), ('bias', tensor([ 0.1134,  0.4575,  0.3052,  0.4959,  0.4295, -0.1042,  0.0697,  0.2338]))])


In [46]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0246], requires_grad=True)
tensor([-0.0246])


In [47]:
net[2].weight.grad == None

True

In [48]:
print(*[(name,param.shape) for name,param in net[0].named_parameters()])
print(*[(name,param.shape) for name,param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [49]:
net.state_dict()['2.bias'].data

tensor([-0.0246])

In [52]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),
                         nn.Linear(8,4),nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}',block1())
    return net
rgnet = nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.0239],
        [0.0240]], grad_fn=<AddmmBackward0>)

In [53]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [54]:
#初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0086,  0.0018, -0.0058, -0.0121]), tensor(0.))

In [55]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [58]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight,42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.6657, -0.3026,  0.2123,  0.4386])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [59]:
#自定义初始化
def my_init(m):
    if type(m) == nn.Linear:
        print("Init",*[(name,param.shape) for name,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)